In [2]:
using JuMP
using CPLEX
#using GLPKMathProgInterface

## THE DIET PROBLEM (CLASSICAL LINEAR PROGRAMMING MODEL)

This model determines a least cost diet which meets the daily
allowances of nutrients for a moderately active man weighing 154 lbs.

We use the classical instance for the diet problem, proposed by:

Dantzig, G B, Chapter 27.1. In Linear Programming and Extensions. Princeton University Press, Princeton, New Jersey, 1963

Stigler's Nutrition model, the formulation is as follows:

$\min \sum_{j\in \mathbb{J}} c_j x_j$

s.t.

$\sum_{j \in \mathbb{J}} a_{ij}x_j \ge b_i, \quad \forall i \in \mathbb{I}$

$x_{j}\ge 0, \quad \forall ~j \in \mathbb{J}$

In [3]:
A=[44.7  1411   2.0   365  0     55.4  33.3  441  0;
    36   897    1.7   99   30.9  17.4  7.9   106  0;
    8.4  422    15.1  9    26    3     23.5  11   60;
    20.6 17     0.6   6    55.8  0.2   0     0    0;
    7.4  448    16.4  19   28.1  0.8   10.3  4    0;
    15.7 661    1     48   0     9.6   8.1   471  0;
    41.7 0      0     0    0.2   0     0.5   5    0;
    2.2  333    0.2   139  169.2 6.4   50.8  316  525;
    4.4  249    0.3   37   0     18.2  3.6   79   0;
    5.8  705    6.8   45   3.5   1     4.9   209  0;
    2.4  138    3.7   80   69    4.3   5.8   37   862;
    2.6  125    4     36   7.2   9     4.5   26   5369;
    5.8  166    3.8   59   16.6  4.7   5.9   21   1184;
    14.3 336    1.8   118  6.7   29.4  7.1   198  2522;
    1.1  106    0     138  918.4 5.7   13.8  33   2755;
    9.6  138    2.7   54   290.7 8.4   5.4   83   1912;
    8.5  87     1.7   173  86.8  1.2   4.3   55   57;
    12.8 99     2.5   154  85.7  3.9   4.3   65   257;
    17.4 1055   3.7   459  5.1   26.9  38.2  93   0;
    26.9 1691   11.4  792  0     38.4  24.6  217  0]

A=A';

b=[30,70,0.8,12,5,1.8,2.7,18,75]

c=ones(20,1);

Creating a range for the coefficient costs

In [4]:
function randomcoef(c)
    cl=zeros(length(c));
    cu=zeros(length(c));
    for i=1:length(c)
        cl[i]=c[i]-rand()/2;
        cu[i]=c[i]+rand()/2;
    end
    return cl,cu
end
c_min,c_max=randomcoef(c)

([0.810384,0.789347,0.912499,0.82459,0.656189,0.670357,0.52806,0.659357,0.720041,0.565775,0.553959,0.583354,0.606064,0.867737,0.600042,0.901423,0.946304,0.510699,0.83475,0.730659],[1.04314,1.04229,1.08797,1.17244,1.16366,1.27251,1.42209,1.13019,1.1377,1.42773,1.06819,1.35642,1.49554,1.40557,1.43377,1.11824,1.359,1.21985,1.3315,1.49041])

From now on, we contruct our Min Max Regret model which is formulated as

$\min_{x} \max_{y,s} \sum_{j \in \mathbb{J}} c^s_j x_j - c^s_j y^s_j$

S.t.

$\sum_{j \in \mathbb{J}} a_{ij}x_j \ge b_i, \quad \forall i \in \mathbb{I}$

$\sum_{j \in \mathbb{J}} a_{ij}y^s_j \ge b_i, \quad \forall i \in \mathbb{i}, \forall s \in \mathbb{S}$

$x_{j}\ge 0, \quad \forall ~j \in \mathbb{J}$

$y^s_{j}\ge 0, \quad \forall ~j \in \mathbb{J}, \forall s \in \mathbb{S}$

$\underline{c}_j\le c^s_j \le \overline{c}_j, \quad \forall ~j \in \mathbb{J}, \forall s \in \mathbb{S}$

In [5]:
function solveLP(A,b,c,solver)
    n=length(c)
    m=size(A,1)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n]>=0)
    @objective(submodel,Min, sum(c[j]*x[j] for j in 1:n) )
    
    @constraint(submodel, req[i = 1:m], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(submodel)
    return valor
end

#Fixing some x
function solveLP(A,b,c,solver,v,max)
    n=length(c)
    m=size(A,1)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n]>=0)
    @objective(submodel,Min, sum(c[j]*x[j] for j in 1:n) )
    
    @constraint(submodel, req[i = 1:m], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    @constraint(submodel, sum(v[j]*x[j] for j in 1:n)==max)
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor,sol=getobjectivevalue(submodel),getvalue(x)
    return valor,sol
end

solveLP (generic function with 2 methods)

Calcularemos los uuper bounds de $x_j$, para ello resolveremos un problema de optimización lineal clásico fijando el valor de $x_j$ en su máximo empírico 

$\hat{x}_j=\max_{i:a_{ij}>0}\Big\{\frac{b_j}{a_{ij}}\Big\},\qquad \forall ~j \in J$

In [6]:
xmax=zeros(length(c));
for j=1:length(c)
    v=zeros(length(c));
    v[j]=1;
    maxim=b[A[:,j].>0]./A[:,j][A[:,j].>0]
    xm=maximum(maxim)
    valor,sol=solveLP(A,b,c,CplexSolver(CPX_PARAM_SCRIND=0),v,xm)
    #validación
    Ar=A[:,setdiff(range(1,size(A,2)),j)]*sol[setdiff(range(1,size(A,2)),j)]
    pos=find(x->x==xm,b./A[:,j])
    pos1=find(x->x==xm,maxim)
    new=xm-Ar[pos]
    if all(maxim[setdiff(range(1,length(maxim)),pos1)].<new)
        xmax[j]=new[1]
    else
        xmax[j]=maximum(maxim[setdiff(range(1,length(maxim)),pos1)])
    end
end

In [7]:
#Subproblem (NP-hard) function 
function SubproblemLP(cup,cun,solver,fix1,fix2,n::Int64,xmax,x,b)
    r=size(A,1)
    m=Model(solver = solver)
    
    @variable(m,yup[1:n]>=0)
    @variable(m,yun[1:n]>=0)
    @variable(m,zup[1:n],Bin)
    @variable(m,zun[1:n],Bin)
    
    @objective(m,Min,sum(cup[j]*yun[j] for j in 1:n) + sum(cun[j]*yup[j] for j in 1:n))
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*x[j] for j in 1:n) + sum(A[i,j]*yup[j] for j in 1:n) - sum(A[i,j]*yup[j] for j in 1:n) >= b[i])
    @constraint(m, plusz[j = 1:n], zup[j]+zun[j]==1)
    @constraint(m, plusy[j = 1:n], zup[j]*(xmax[j]-x[j]) >= yup[j])
    @constraint(m, minusy[j = 1:n], zun[j]*x[j] >= yun[j])
    for j=1:n
        if fix1[j]==1
            @constraint(n, zup[j]==1)
        else
            if fix2[j]==1
                @constraint(n, zun[j]==1)
            end
        end
    end
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(m)
    zu=getvalue(zup)
    cost_scen=zeros(Float64,n)
    for i=1:n
        if zu[i]==1
            cost_scen[j]=cun[j]
        else
            cost_scen[j]=cup[j]
        end
    end    
    return valor,cost_scen
end

SubproblemLP (generic function with 1 method)

In [8]:
function solveRLP(A,b,cminus, cplus, xmax, n::Int64, solver=CplexSolver())
    r=size(A,1)
    m = Model(solver = solver)
    
    @variable(m, x[1:n]>=0)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, req[i = 1:r], sum(A[i,j]*x[j] for j in 1:n) >= b[i])
    
    
    cMidPoint=zeros(Float64,n)
    for i in 1:n
        cMidPoint[i]=(cminus[i]+cplus[i])/2
    end
    optimalCost = solveLP(A,b,cMidPoint,CplexSolver(CPX_PARAM_SCRIND=0))
    rhs = - optimalCost 
    for i in 1:n
        rhs += cMidPoint[i]*x[i]
    end
    @constraint(m,θ >=rhs)        
    #print(m)

    tt = 0.0
    called = 0.0
    separationtime=0.0
    separated=0
    
    
    function lazyGenerator(m)
        #print("\n\n entro en generador de cortes")
        EPSILON=0.00001
        tt = time()
        called += 1
        x_val = getvalue(x)
                
        fixup=zeros(Int64,n)
        fixun=zeros(Int64,n)
        for i in 1:n
            if x_val[i]<EPSILON 
                fixun[i]=1
            else
                if x_val[i]<(xmax-EPSILON)
                    fixup[i]=1
                end
            end
        end
        println("\t\t",fixup,"\n")
        println("\t\t",fixun,"\n")
        optimalCost,cost = SubproblemLP(cplus,cminus,CplexSolver(CPX_PARAM_SCRIND=0),fixun,fixup,n,xmax,x,b)
        #println("\t\t",optimalCost,"\n")
       
        rhs = - optimalCost 
        for i in 1:n
            rhs += cost[i]*x[i]
        end
        #println(rhs)
        #print("*")
        @lazyconstraint(m, θ >= rhs )
        separated += 1
        separationtime += time()-tt
    end
    addlazycallback(m, lazyGenerator) #es necesario un corte lazy ya que le indica al programa que no la he incluido
    print(m)
    status = solve(m)
    println(status)
    println("maxRegret: ",getobjectivevalue(m))
    println("maxRegret: ",getvalue(x))
end

solveRLP (generic function with 2 methods)

In [9]:
solveRLP(A,b,c_min, c_max, xmax, length(c))

Min θ
Subject to
 44.7 x[1] + 36 x[2] + 8.4 x[3] + 20.6 x[4] + 7.4 x[5] + 15.7 x[6] + 41.7 x[7] + 2.2 x[8] + 4.4 x[9] + 5.8 x[10] + 2.4 x[11] + 2.6 x[12] + 5.8 x[13] + 14.3 x[14] + 1.1 x[15] + 9.6 x[16] + 8.5 x[17] + 12.8 x[18] + 17.4 x[19] + 26.9 x[20] ≥ 30
 1411 x[1] + 897 x[2] + 422 x[3] + 17 x[4] + 448 x[5] + 661 x[6] + 333 x[8] + 249 x[9] + 705 x[10] + 138 x[11] + 125 x[12] + 166 x[13] + 336 x[14] + 106 x[15] + 138 x[16] + 87 x[17] + 99 x[18] + 1055 x[19] + 1691 x[20] ≥ 70
 2 x[1] + 1.7 x[2] + 15.1 x[3] + 0.6 x[4] + 16.4 x[5] + x[6] + 0.2 x[8] + 0.3 x[9] + 6.8 x[10] + 3.7 x[11] + 4 x[12] + 3.8 x[13] + 1.8 x[14] + 2.7 x[16] + 1.7 x[17] + 2.5 x[18] + 3.7 x[19] + 11.4 x[20] ≥ 0.8
 365 x[1] + 99 x[2] + 9 x[3] + 6 x[4] + 19 x[5] + 48 x[6] + 139 x[8] + 37 x[9] + 45 x[10] + 80 x[11] + 36 x[12] + 59 x[13] + 118 x[14] + 138 x[15] + 54 x[16] + 173 x[17] + 154 x[18] + 459 x[19] + 792 x[20] ≥ 12
 30.9 x[2] + 26 x[3] + 55.8 x[4] + 28.1 x[5] + 0.2 x[7] + 169.2 x[8] + 3.5 x[10] + 69 x[11] + 7.2 